In [3]:
import pandas as pd
import geopandas as gpd
import folium

import ee
import geemap
import math
from glob import glob

In [4]:
import sys
print(sys.executable)

c:\Anaconda3\python.exe


### **LOADING AND EXPLORING DENGUE EXCEL DATA**

This section loads the raw dengue dataset stored in an Excel file containing multiple sheets (each representing a year of reported cases).
We preview one sheet (e.g., Dengue 2018) to understand its structure and verify data consistency across years.

In [5]:
excel_file = pd.ExcelFile("../data/raw/Dengue.xlsx")
sheet_names = excel_file.sheet_names
print(sheet_names)

['Dengue 2018', 'Dengue 2019', 'Dengue 2020', 'Dengue 2021', 'Dengue 2022', 'Dengue 2023', 'Dengue 2024', 'Dengue 2025']


In [6]:
sample_sheet = pd.read_excel(excel_file, sheet_name='Dengue 2018')
sample_sheet.head()

,Streetpurok,Age,Sex,DOB,DAdmit,DOnset,Outcome,Year,NameOfDru,Barangay
0,"VILLAFLORA, CUGMAN, CAGAYAN DE ORO CITY",12.0,F,30/12/2005 0:00,2018-11-11 00:00:00,2018-07-11 00:00:00,A,2018,DEL MONTE BUGO CLINIC,CUGMAN
1,ZONE 10,5.0,F,2013-05-07 00:00:00,16/11/2018 0:00,2018-11-11 00:00:00,A,2018,CAGAYAN DE ORO MATERNITY CHILDREN'S HOSP. AND ...,CARMEN
2,NaN,5.0,M,2013-04-09 00:00:00,18/11/2018 0:00,15/11/2018 0:00,A,2018,CAGAYAN DE ORO MEDICAL CENTER,LAPASAN
3,NaN,9.0,M,20/05/2009 0:00,22/11/2018 0:00,20/11/2018 0:00,A,2018,CAGAYAN DE ORO MEDICAL CENTER,LAPASAN
4,3RD ST. SAN PEDRO VILLAGE,5.0,F,2013-06-11 00:00:00,18/11/2018 0:00,15/11/2018 0:00,A,2018,MARIA REYNA HOSPITAL,GUSA


### **PROCESSING AND AGGREGATING DENGUE CASES BY WEEK AND BARANGAY**

Here we define a data cleaning and aggregation function to standardize barangay names and count dengue cases per week for each barangay.
We apply this function to all yearly sheets, then combine them into a single DataFrame with columns: Barangay, Week, Year, and Cases. After processing all sheets, we concatenate them into one unified dataset.

In [7]:
def count_dengue_cases_by_week_barangay(df):
    # Ensure 'DAdmit' is datetime, coercing errors
    df["DAdmit"] = pd.to_datetime(df["DAdmit"], dayfirst=True, errors="coerce")

    # Extract ISO week number (1–53)
    df['Week'] = df['DAdmit'].dt.isocalendar().week

    # Group and count cases per Barangay-Week pair
    dengue_counts = df.groupby(['Barangay', 'Week']).size().reset_index(name='Cases')

    # Create full range of weeks (1–53) and all unique barangays
    all_barangays = df['Barangay'].unique()
    all_weeks = range(1, 54)

    # Create a MultiIndex of all possible combinations
    full_index = pd.MultiIndex.from_product([all_barangays, all_weeks], names=['Barangay', 'Week'])

    # Reindex and fill missing combinations with 0
    dengue_counts = dengue_counts.set_index(['Barangay', 'Week']).reindex(full_index, fill_value=0).reset_index()

    # Clean and format barangay names
    dengue_counts['Barangay'] = dengue_counts['Barangay'].str.strip().str.title()
    dengue_counts['Barangay'] = dengue_counts['Barangay'].apply(lambda x: 'Barangay ' + x if str(x).isdigit() else x)
    dengue_counts['Barangay'] = dengue_counts['Barangay'].str.replace(r'\s*\(Pob\.\)\s*', '', regex=True)

    # Drop any remaining nulls
    dengue_counts = dengue_counts.dropna()

    return dengue_counts

# Apply to all sheets
all_years_data = []
for sheet_name in sheet_names:
    df_sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
    dengue_counts_sheet = count_dengue_cases_by_week_barangay(df_sheet.copy())
    all_years_data.append(dengue_counts_sheet)

# Add 'Year' column to each dataset
for i, sheet_name in enumerate(sheet_names):
    year = int(sheet_name.split(' ')[1])
    all_years_data[i]['Year'] = year

# Combine all into one DataFrame
combined_dengue_counts = pd.concat(all_years_data, ignore_index=True)
display(combined_dengue_counts)

,Barangay,Week,Cases,Year
0,Cugman,1,0,2018
1,Cugman,2,2,2018
2,Cugman,3,0,2018
3,Cugman,4,1,2018
4,Cugman,5,2,2018
...,...,...,...,...
24905,Dansolihon,49,0,2025
24906,Dansolihon,50,0,2025
24907,Dansolihon,51,0,2025
24908,Dansolihon,52,0,2025


### **INTEGRATING GEOSPATIAL BARANGAY BOUNDARIES (CDO SHAPEFILE)**

This step loads the Cagayan de Oro barangay shapefile using GeoPandas and prepares it for spatial mapping (EPSG:4326 projection).
We visualize barangay boundaries using Folium and ensure that barangay names from the dengue dataset match those in the shapefile.

It also includes name reconciliation, where mismatched or abbreviated barangay names are corrected using a custom mapping dictionary. Finally, we verify that only barangays present in the shapefile remain in the dataset.

In [8]:
ph_barangays = gpd.read_file("../data/raw/PH_Adm4_BgySubMuns.zip")
display(ph_barangays.head())

,adm1_psgc,adm2_psgc,adm3_psgc,adm4_psgc,adm4_en,geo_level,len_crs,area_crs,len_km,area_km2,geometry
0,100000000,102800000,102801000,102801001,Adams,Bgy,45997,111184551,45,111.0,"POLYGON ((280486.139 2048388.148, 283183.575 2..."
1,100000000,102800000,102802000,102802001,Bani,Bgy,5982,1761135,5,1.0,"POLYGON ((247619.019 2022359.605, 247623.585 2..."
2,100000000,102800000,102802000,102802002,Buyon,Bgy,9117,3875134,9,3.0,"POLYGON ((249124.075 2019071.573, 249782.73 20..."
3,100000000,102800000,102802000,102802003,Cabaruan,Bgy,7745,2987648,7,2.0,"POLYGON ((245180.248 2021560.567, 246674.214 2..."
4,100000000,102800000,102802000,102802004,Cabulalaan,Bgy,4502,1018354,4,1.0,"POLYGON ((244550.24 2023086.477, 244585.568 20..."


In [9]:
cdo_barangays = ph_barangays[ph_barangays['adm3_psgc'] == 1030500000] # CDO Correspondence Code
cdo_barangays = cdo_barangays[['adm4_en', 'geometry']].rename(columns={'adm4_en': 'barangay'})
cdo_barangays = cdo_barangays.to_crs(epsg=4326) # Convert to EPSG Code 4326
display(cdo_barangays)

,barangay,geometry
31368,Agusan,"POLYGON ((124.74786 8.4978, 124.74784 8.49732,..."
31369,Baikingon,"POLYGON ((124.59103 8.48108, 124.59155 8.48025..."
31370,Bulua,"POLYGON ((124.61392 8.52148, 124.61408 8.52131..."
31371,Balubal,"POLYGON ((124.76861 8.48196, 124.76911 8.482, ..."
31372,Balulang,"POLYGON ((124.63114 8.46818, 124.6313 8.46754,..."
...,...,...
31443,Barangay 29,"POLYGON ((124.64835 8.48363, 124.64931 8.48318..."
31444,Barangay 31,"POLYGON ((124.65738 8.4831, 124.65757 8.48275,..."
31445,Barangay 35,"POLYGON ((124.65441 8.47853, 124.65449 8.47851..."
31446,Barangay 36,"POLYGON ((124.65592 8.47701, 124.65624 8.47695..."


In [10]:
# Create base map centered on CDO
map_cdo = folium.Map(location=[8.4803, 124.6498], zoom_start=10)

# Add barangays to the map
folium.GeoJson(
    cdo_barangays,
    name='Barangays',
    tooltip=folium.features.GeoJsonTooltip(
        fields=['barangay'],
        aliases=['Barangay:'],
        localize=True
    )
).add_to(map_cdo)
map_cdo

In [11]:
# Get the unique barangay names from the processed dengue data
unique_barangays_in_data = combined_dengue_counts['Barangay'].unique()

# Get the barangay names from the shapefile data
unique_barangays_in_shapefile = cdo_barangays['barangay'].unique()

# Find barangays in the dengue data that are not in the shapefile data
barangays_only_in_data = sorted(set(unique_barangays_in_data) - set(unique_barangays_in_shapefile))

# Find barangays in the shapefile data that are not in the dengue data
barangays_only_in_shapefile = sorted(set(unique_barangays_in_shapefile) - set(unique_barangays_in_data))

print("[INFO] Barangays in the dengue data but not in the shapefile data:")
for barangay in barangays_only_in_data:
    print(f"- {barangay}")

print("\n[INFO] Barangays in the shapefile data but not in the dengue data:")
for barangay in barangays_only_in_shapefile:
    print(f"- {barangay}")

[INFO] Barangays in the dengue data but not in the shapefile data:
- Baya
- Buki
- Camaman-An
- Canito-An
- Canitoan
- Carm
- Cor
- Corr
- F.S. Catanico
- Haye
- Maca
- Mont
- Naza
- Pata
- Pigsag-An
- Poblacion
- Tabl
- Tago
- Terr
- Zone

[INFO] Barangays in the shapefile data but not in the dengue data:
- Barangay 39
- Barangay 8
- Barangay 9
- Camaman-an
- Canito-an
- Pigsag-an


In [12]:
# Create a mapping for inconsistent barangay names based on the comparison
barangay_mapping = {
    'Baya': 'Bayabas', # Assuming 'Baya' is an abbreviation for 'Bayabas'
    'Buki': None, # 'Buki' is not a known barangay in CDO. Needs clarification or removal.
    'Camaman-An': 'Camaman-an', # Correcting capitalization
    'Canito-An': 'Canito-an', # Correcting capitalization
    'Canitoan': 'Canito-an', # Correcting hyphenation
    'Carm': 'Carmen', # Assuming 'Carm' is an abbreviation for 'Carmen'
    'Cor': 'Corrales', # Assuming 'Cor' is an abbreviation for 'Corrales'
    'Corr': 'Corrales', # Assuming 'Corr' is an abbreviation for 'Corrales'
    'F.S. Catanico': 'F. S. Catanico', # Correcting spacing
    'Haye': 'Hayes', # Assuming 'Haye' is a typo for 'Hayes'
    'Maca': 'Macasandig', # Assuming 'Maca' is an abbreviation for 'Macasandig'
    'Mont': 'Montalban', # Assuming 'Mont' is an abbreviation for 'Montalban'
    'Naza': 'Nazareth', # Assuming 'Naza' is an abbreviation for 'Nazareth'
    'Pata': 'Patag', # Assuming 'Pata' is an abbreviation for 'Patag'
    'Pigsag-An': 'Pigsag-an', # Correcting capitalization
    'Poblacion': None, # 'Poblacion' refers to multiple barangays in the city proper. Needs to be mapped to the specific barangay or handled as unknown.
    'Tabl': 'Tablon', # Assuming 'Tabl' is an abbreviation for 'Tablon'
    'Tago': None, # Assuming 'Tago' is an abbreviation for 'Tagoloan'. Note: Tagoloan is a municipality, not a barangay in CDO. This might need further clarification or removal if it's an error in the source data.
    'Terr': None, # Assuming 'Terr' is an abbreviation for 'Terry Hills'. Note: Terry Hills is a subdivision/area, not a barangay. This might need further clarification or removal.
    'Zone': None, # 'Zone' is too generic, likely needs to be mapped to a specific barangay or handled as unknown.
}

# Apply the mapping to the 'Barangay' column
combined_dengue_counts['Barangay'] = combined_dengue_counts['Barangay'].replace(barangay_mapping)

# Remove rows where Barangay is None after mapping (for generic or unknown entries)
combined_dengue_counts.dropna(subset=['Barangay'], inplace=True)

# Filter the combined dengue data to only include barangays present in the shapefile data
combined_dengue_counts = combined_dengue_counts[combined_dengue_counts['Barangay'].isin(cdo_barangays['barangay'].unique())].copy()

# Re-apply the title case and "Barangay " prefixing after mapping
combined_dengue_counts['Barangay'] = combined_dengue_counts['Barangay'].apply(lambda x: 'Barangay ' + x if str(x).isdigit() else x)

# Recount unique barangays to verify
unique_barangays_count_after_mapping = combined_dengue_counts['Barangay'].nunique()
print(f"[INFO] Number of unique barangays after mapping and filtering: {unique_barangays_count_after_mapping}")

[INFO] Number of unique barangays after mapping and filtering: 77


In [13]:
# Get the unique barangay names from the processed dengue data after mapping
unique_barangays_in_data_after_mapping = combined_dengue_counts['Barangay'].unique()

# Get the barangay names from the shapefile data
unique_barangays_in_shapefile = cdo_barangays['barangay'].unique()

# Find barangays in the shapefile data that are not in the processed dengue data (after mapping)
barangays_only_in_shapefile_after_mapping = sorted(set(unique_barangays_in_shapefile) - set(unique_barangays_in_data_after_mapping))

print("[INFO] Barangays in the shapefile data but not in the processed dengue data after mapping:")
for barangay in barangays_only_in_shapefile_after_mapping:
    print(f"- {barangay}")

[INFO] Barangays in the shapefile data but not in the processed dengue data after mapping:
- Barangay 39
- Barangay 8
- Barangay 9


### **COMPLETING MISSING BARANGAYS AND TEMPORAL COVERAGE**

To ensure uniformity across the dataset, we fill missing barangay-week-year combinations with zero dengue cases.
This guarantees that each barangay (including those with no recorded cases) appears consistently from January 2018 to December 2024.

We also apply a temporal filter to limit the data within this range.

In [14]:
unique_years = combined_dengue_counts['Year'].unique()
all_weeks = range(1, 54)  # ISO weeks (1–53)

placeholder_data = []
for barangay in barangays_only_in_shapefile_after_mapping:
    for year in unique_years:
        for week in all_weeks:
            placeholder_data.append({
                'Barangay': barangay,
                'Year': year,
                'Week': week,
                'Cases': 0
            })

placeholder_df = pd.DataFrame(placeholder_data)
display(placeholder_df)

,Barangay,Year,Week,Cases
0,Barangay 39,2018,1,0
1,Barangay 39,2018,2,0
2,Barangay 39,2018,3,0
3,Barangay 39,2018,4,0
4,Barangay 39,2018,5,0
...,...,...,...,...
1267,Barangay 9,2025,49,0
1268,Barangay 9,2025,50,0
1269,Barangay 9,2025,51,0
1270,Barangay 9,2025,52,0


In [15]:
combined_dengue_counts_with_placeholders = pd.concat([combined_dengue_counts, placeholder_df], ignore_index=True)
display(combined_dengue_counts_with_placeholders)

,Barangay,Week,Cases,Year
0,Cugman,1,0,2018
1,Cugman,2,2,2018
2,Cugman,3,0,2018
3,Cugman,4,1,2018
4,Cugman,5,2,2018
...,...,...,...,...
25541,Barangay 9,49,0,2025
25542,Barangay 9,50,0,2025
25543,Barangay 9,51,0,2025
25544,Barangay 9,52,0,2025


In [16]:
# Reset index to flatten any hierarchical indexing from earlier steps
combined_dengue_counts_with_placeholders = combined_dengue_counts_with_placeholders.reset_index(drop=True)

# Remove duplicate rows based on Barangay, Year, and Week
combined_dengue_counts_with_placeholders = combined_dengue_counts_with_placeholders.drop_duplicates(
    subset=['Barangay', 'Year', 'Week']
)

# Get unique barangays and years present in the dataset
unique_barangays = combined_dengue_counts_with_placeholders['Barangay'].unique()
unique_years = combined_dengue_counts_with_placeholders['Year'].unique()
all_weeks = range(1, 54)  # ISO weeks 1–53

# Create a complete index of all combinations of Barangay, Year, and Week
full_index = pd.MultiIndex.from_product(
    [unique_barangays, unique_years, all_weeks],
    names=['Barangay', 'Year', 'Week']
)

# Reindex dataset to include all combinations, filling missing values with 0
combined_dengue_counts_with_placeholders = combined_dengue_counts_with_placeholders.set_index(
    ['Barangay', 'Year', 'Week']
).reindex(full_index, fill_value=0)

# Reset index but keep the Barangay, Year, Week columns
combined_dengue_counts_with_placeholders = combined_dengue_counts_with_placeholders.reset_index()

# Sort the data for readability and consistency
combined_dengue_counts_with_placeholders = combined_dengue_counts_with_placeholders.sort_values(
    by=['Year', 'Barangay', 'Week']
).reset_index(drop=True)

# Make a copy of the cleaned dataframe
final_dengue_counts = combined_dengue_counts_with_placeholders.copy()

# Filter from Week 1 of 2018 to Week 53 of 2024, excluding Week 1 of 2025
def filter_data(df):
    # Create a 'Date' column representing the Monday of each ISO week
    df["Date"] = pd.to_datetime(df["Year"].astype(str) + df["Week"].astype(str) + '1', format='%G%V%u', errors='coerce')
    
    # Define range
    start_date = "2018-01-01"
    end_date = "2024-12-31"
    
    # Filter by date range
    filtered = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]
    
    # Explicitly remove Week 1 of 2025 (if it appears due to ISO overlap)
    filtered = filtered[~((filtered["Year"] == 2025) & (filtered["Week"] == 1))]
    
    # Drop the helper column if not needed
    filtered = filtered.drop(columns=["Date"])
    return filtered

final_dengue_counts = filter_data(final_dengue_counts)

# Display final dataframe
display(final_dengue_counts)

# Print number of unique barangays to verify correctness
print(f"[INFO] Number of unique barangays after ensuring all shapefile barangays are included: {final_dengue_counts['Barangay'].nunique()}")

,Barangay,Year,Week,Cases
0,Agusan,2018,1,0
1,Agusan,2018,2,5
2,Agusan,2018,3,2
3,Agusan,2018,4,1
4,Agusan,2018,5,1
...,...,...,...,...
29675,Tumpagon,2024,49,0
29676,Tumpagon,2024,50,0
29677,Tumpagon,2024,51,0
29678,Tumpagon,2024,52,0


[INFO] Number of unique barangays after ensuring all shapefile barangays are included: 80


### **CREATING A GEODATAFRAME FOR SPATIAL ANALYSIS**

Here we merge the cleaned dengue case data with barangay geometry from the shapefile to create a GeoDataFrame.
This format is essential for spatial operations, visualization, and integration with Earth Engine.

In [17]:
# Merge the dengue case data with the barangay geometry data
final_dengue_counts = pd.merge(
    final_dengue_counts,
    cdo_barangays,
    left_on='Barangay',
    right_on='barangay',
    how='left'  # Use a left merge to keep all dengue data and add geometry where available
)

# Drop the redundant 'barangay' column from the merge
final_dengue_counts = final_dengue_counts.drop(columns=['barangay'])
final_dengue_counts = final_dengue_counts.rename(columns={'geometry': 'Geometry'})

# Convert the merged DataFrame to a GeoDataFrame
final_dengue_counts_gdf = gpd.GeoDataFrame(
    final_dengue_counts, geometry='Geometry', crs='EPSG:4326'
)

display(final_dengue_counts_gdf)

,Barangay,Year,Week,Cases,Geometry
0,Agusan,2018,1,0,"POLYGON ((124.74786 8.4978, 124.74784 8.49732,..."
1,Agusan,2018,2,5,"POLYGON ((124.74786 8.4978, 124.74784 8.49732,..."
2,Agusan,2018,3,2,"POLYGON ((124.74786 8.4978, 124.74784 8.49732,..."
3,Agusan,2018,4,1,"POLYGON ((124.74786 8.4978, 124.74784 8.49732,..."
4,Agusan,2018,5,1,"POLYGON ((124.74786 8.4978, 124.74784 8.49732,..."
...,...,...,...,...,...
29675,Tumpagon,2024,49,0,"POLYGON ((124.46685 8.34849, 124.46816 8.34767..."
29676,Tumpagon,2024,50,0,"POLYGON ((124.46685 8.34849, 124.46816 8.34767..."
29677,Tumpagon,2024,51,0,"POLYGON ((124.46685 8.34849, 124.46816 8.34767..."
29678,Tumpagon,2024,52,0,"POLYGON ((124.46685 8.34849, 124.46816 8.34767..."


### **INTEGRATING CLIMATE DATA FROM GOOGLE EARTH ENGINE (GEE)**

This final stage connects the cleaned spatial dengue dataset to Google Earth Engine (GEE) to extract relevant climate variables such as:

- Temperature
- Dewpoint and Relative Humidity
- Precipitation

Each barangay-month record is converted to a GEE FeatureCollection, then mapped with climate features from ERA5 and CHIRPS datasets.
The enhanced dataset is exported annually as CSV for further modeling.

In [18]:
# Path to the service account key JSON
SERVICE_ACCOUNT_JSON = "../service-account.json"
SERVICE_ACCOUNT_EMAIL = "denvue-team@denvue-ds.iam.gserviceaccount.com"

# Authenticate with the service account
credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT_EMAIL, SERVICE_ACCOUNT_JSON)
ee.Initialize(credentials, project="denvue-ds")
print("Earth Engine initialized successfully!")

Earth Engine initialized successfully!


In [19]:
# Convert GeoDataFrame to Earth Engine FeatureCollection
def gdf_to_fc_chunked(gdf, batch_size=300):
    """Convert a GeoDataFrame into multiple FeatureCollections under 10 MB each."""
    batches = []
    for i in range(0, len(gdf), batch_size):
        chunk = gdf.iloc[i:i + batch_size]
        features = []
        for _, row in chunk.iterrows():
            geom = ee.Geometry(row["Geometry"].__geo_interface__)
            start_date = ee.Date.fromYMD(int(row["Year"]), 1, 1).advance(int(row["Week"]) - 1, "week")
            feature = ee.Feature(geom, {
                "Barangay": row["Barangay"],
                "Year": int(row["Year"]),
                "Week": int(row["Week"]),
                "Cases": int(row["Cases"])
            }).set("system:time_start", start_date.millis())
            features.append(feature)
        batches.append(ee.FeatureCollection(features))
    return batches

In [20]:
# ERA5 for temperature & humidity
era5 = ee.ImageCollection("ECMWF/ERA5/MONTHLY")

# CHIRPS for precipitation
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")

In [21]:
def get_weekly_collection(collection, year, week):
    # Convert to ee.Number (in case they’re Python ints)
    year = ee.Number(year)
    week = ee.Number(week)
    
    # Compute start (Monday) and end (next Monday) of the ISO week
    start_date = ee.Date.fromYMD(year, 1, 1).advance(week.subtract(1), "week")
    end_date = start_date.advance(1, "week")

    col = collection.filterDate(start_date, end_date)
    
    return ee.ImageCollection(
        ee.Algorithms.If(
            col.size().gt(0),
            col,  # Use this week
            collection.filterDate("1900-01-01", start_date)
                      .sort("system:time_start", False)
                      .limit(1)  # Fallback to last available image
                      .map(lambda img: img)  # Wrap as ImageCollection
        )
    )

def add_climate_features(feature):
    year = ee.Number(feature.get("Year")).int()
    week = ee.Number(feature.get("Week")).int()
    geom = feature.geometry()

    # Get collections for this week
    era5_col = get_weekly_collection(era5, year, week)
    chirps_col = get_weekly_collection(chirps, year, week)

    # Collapse to single images
    era5_img = era5_col.mean()     # average across days in the week
    chirps_mean = chirps_col.mean()  # average weekly precipitation

    # Temperature (K → °C)
    temp_c = ee.Number(
        era5_img.select("mean_2m_air_temperature")
        .reduceRegion(ee.Reducer.mean(), geom, 1000, maxPixels=1e13)
        .get("mean_2m_air_temperature")
    ).subtract(273.15)

    # Dewpoint (K → °C)
    dew_c = ee.Number(
        era5_img.select("dewpoint_2m_temperature")
        .reduceRegion(ee.Reducer.mean(), geom, 1000, maxPixels=1e13)
        .get("dewpoint_2m_temperature")
    ).subtract(273.15)

    # Relative Humidity
    a = ee.Number(17.625)
    b = ee.Number(243.04)
    alpha_t = a.multiply(temp_c).divide(temp_c.add(b))
    alpha_d = a.multiply(dew_c).divide(dew_c.add(b))
    humidity = ee.Number(100).multiply(alpha_d.exp().divide(alpha_t.exp()))

    # Precipitation (mm/day average for that week)
    precip = chirps_mean.reduceRegion(
        ee.Reducer.mean(), geom, 5000, maxPixels=1e13
    ).get("precipitation")

    return feature.set({
        "Temperature": temp_c,
        "Humidity": humidity,
        "Precipitation": precip,
    })

In [22]:
# Split GDF by year first, then batch upload within each year
for y in range(2018, 2025):
    gdf_y = final_dengue_counts_gdf[final_dengue_counts_gdf["Year"] == y]
    batches = gdf_to_fc_chunked(gdf_y, batch_size=300)

    for i, fc_batch in enumerate(batches):
        print(f"[INFO] Processing Year {y}, Batch {i+1}/{len(batches)}")
        fc_with_climate_batch = fc_batch.map(add_climate_features)

        # Define half-year date ranges
        start_H1 = ee.Date.fromYMD(y, 1, 1)
        end_H1 = ee.Date.fromYMD(y, 7, 1)
        start_H2 = ee.Date.fromYMD(y, 7, 1)
        end_H2 = ee.Date.fromYMD(y + 1, 1, 1)

        # Export H1
        subset_1 = fc_with_climate_batch.filterDate(start_H1, end_H1)
        filename_1 = f"../data/processed/dengue_climate_{y}_H1_batch{i+1}.csv"
        geemap.ee_to_csv(subset_1, filename=filename_1)
        print(f"[INFO] Exported {filename_1}")

        # Export H2
        subset_2 = fc_with_climate_batch.filterDate(start_H2, end_H2)
        filename_2 = f"../data/processed/dengue_climate_{y}_H2_batch{i+1}.csv"
        geemap.ee_to_csv(subset_2, filename=filename_2)
        print(f"[INFO] Exported {filename_2}")

[INFO] Processing Year 2018, Batch 1/15
[INFO] Exported ../data/processed/dengue_climate_2018_H1_batch1.csv
[INFO] Exported ../data/processed/dengue_climate_2018_H2_batch1.csv
[INFO] Processing Year 2018, Batch 2/15
[INFO] Exported ../data/processed/dengue_climate_2018_H1_batch2.csv
[INFO] Exported ../data/processed/dengue_climate_2018_H2_batch2.csv
[INFO] Processing Year 2018, Batch 3/15
[INFO] Exported ../data/processed/dengue_climate_2018_H1_batch3.csv
[INFO] Exported ../data/processed/dengue_climate_2018_H2_batch3.csv
[INFO] Processing Year 2018, Batch 4/15
[INFO] Exported ../data/processed/dengue_climate_2018_H1_batch4.csv
[INFO] Exported ../data/processed/dengue_climate_2018_H2_batch4.csv
[INFO] Processing Year 2018, Batch 5/15
[INFO] Exported ../data/processed/dengue_climate_2018_H1_batch5.csv
[INFO] Exported ../data/processed/dengue_climate_2018_H2_batch5.csv
[INFO] Processing Year 2018, Batch 6/15
[INFO] Exported ../data/processed/dengue_climate_2018_H1_batch6.csv
[INFO] Expor

In [23]:
# Merge all batched half-yearly exports into one DataFrame
files = sorted(glob("../data/processed/dengue_climate_*_H*_batch*.csv"))
print(f"[INFO] Found {len(files)} files to merge.")

dfs = []
for f in files:
    try:
        df_part = pd.read_csv(f)
        if not df_part.empty:
            dfs.append(df_part)
        else:
            print(f"[WARN] Skipping empty file: {f}")
    except pd.errors.EmptyDataError:
        print(f"[WARN] Skipping unreadable file: {f}")

# Merge all parts
df = pd.concat(dfs, ignore_index=True)
print(f"[INFO] Merged DataFrame shape: {df.shape}")

# Sort by Year and Week (weekly dataset)
if {"Year", "Week"}.issubset(df.columns):
    df = df.sort_values(by=["Year", "Week"]).reset_index(drop=True)
elif {"Year", "Month"}.issubset(df.columns):
    df = df.sort_values(by=["Year", "Month"]).reset_index(drop=True)

print(f"[INFO] Final DataFrame shape: {df.shape}")
df.head()

[INFO] Found 210 files to merge.
[INFO] Merged DataFrame shape: (29680, 7)
[INFO] Final DataFrame shape: (29680, 7)


,Barangay,Cases,Humidity,Precipitation,Temperature,Week,Year
0,Agusan,0,85.035175,6.717665,25.657637,1,2018
1,Baikingon,0,84.282708,5.530006,26.329614,1,2018
2,Balubal,1,85.548596,6.634705,25.205957,1,2018
3,Balulang,1,84.282708,6.214297,26.329614,1,2018
4,Barangay 1,0,84.282708,7.585130,26.329614,1,2018


In [24]:
# Check for null values
df.isnull().sum()

Barangay             0
Cases                0
Humidity             0
Precipitation    11130
Temperature          0
Week                 0
Year                 0
dtype: int64

In [25]:
# Fill in null values
df["Precipitation"] = df["Precipitation"].fillna(0)

In [26]:
# Export as CSV
df.to_csv("../data/processed/dengue_climate_all_years.csv", index=False)
print("[INFO] Merged all years to ../data/processed/dengue_climate_all_years.csv")

[INFO] Merged all years to ../data/processed/dengue_climate_all_years.csv


**ADDING POPULATION DATA**

In [32]:
import os
from pathlib import Path

print("CWD:", os.getcwd())
print("raw contents:", [p.name for p in Path("data/raw").glob("*")])


CWD: f:\4th year\capstone\NEW NOTEBOOKS
raw contents: ['cdo_population.csv', 'Dengue.xlsx', 'PH_Adm4_BgySubMuns.zip']


In [34]:
dengue_data = pd.read_csv("data/processed/dengue_climate_all_years.csv")
pop_data    = pd.read_csv("data/raw/cdo_population.csv")

print(dengue_data.columns)
print(pop_data.columns)

Index(['Barangay', 'Cases', 'Humidity', 'Precipitation', 'Temperature', 'Week',
       'Year'],
      dtype='object')
Index(['Barangay', 'Population'], dtype='object')


In [35]:
dengue_data = dengue_data.merge(
    pop_data[["Barangay", "Population"]],
    on="Barangay",
    how="left"
)
dengue_data

,Barangay,Cases,Humidity,Precipitation,Temperature,Week,Year,Population
0,Agusan,0,85.035175,6.717665,25.657637,1,2018,"19,039"
1,Baikingon,0,84.282708,5.530006,26.329614,1,2018,"2,879"
2,Balubal,1,85.548596,6.634705,25.205957,1,2018,"7,013"
3,Balulang,1,84.282708,6.214297,26.329614,1,2018,"42,205"
4,Barangay 1,0,84.282708,7.585130,26.329614,1,2018,168
...,...,...,...,...,...,...,...,...
29675,Bayanga,0,82.775006,5.250069,27.049707,53,2024,"3,402"
29676,Besigan,0,86.612443,5.292381,26.140588,53,2024,"1,700"
29677,Bonbon,0,80.762922,0.000000,27.361536,53,2024,"10,976"
29678,Bugo,0,84.744716,5.170116,26.226479,53,2024,"31,229"


In [36]:
# Export as CSV
dengue_data.to_csv("../data/processed/dengue_climate_all_years.csv", index=False)
print("[INFO] Merged dengue and population data to ../data/processed/dengue_climate_all_years.csv")

[INFO] Merged dengue and population data to ../data/processed/dengue_climate_all_years.csv
